In [1]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from textblob import TextBlob
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import faiss
from sentence_transformers import SentenceTransformer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.sequence import pad_sequences

nlp = spacy.load("en_core_web_sm")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 14:54:48.279764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv("Datasets/preprocessed.csv")
data

/var/folders/qq/g1fc9b_10d1gd8z3wl4n208w0000gn/T/ipykernel_28648/1752640760.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Datasets/preprocessed.csv")


,Content,Label,Content_int,Cleaned_Content,Preprocessed_Content
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,...",denial of normal the con be asked to comment o...,denial normal con ask comment tragedy emotiona...
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22...",just by being able to tweet this insufferable ...,able tweet insufferable bullshit proves trump ...
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,...",that is retarded you too cute to be single tha...,retard cute single life
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,...",thought of a real badass mongol style declarat...,thought real badass mongol style declaration w...
4,afro american basho,1,"[146715, 46, 47, 48, 146714]",afro american basho,afro american basho
...,...,...,...,...,...
440901,crash another movie from left field i have to ...,0,"[146715, 1780, 649, 1673, 347, 701, 1702, 80, ...",crash another movie from left field i have to ...,crash movie left field ask star war episode ii...
440902,i why do not you debate first before starting ...,0,"[146715, 80, 186, 340, 76, 26, 6407, 566, 2608...",i why do not you debate first before starting ...,debate start edit war
440903,removal of i reverted the removal of the above...,0,"[146715, 18731, 1, 80, 48182, 3, 18731, 1, 3, ...",removal of i reverted the removal of the above...,removal revert removal section believe misguid...
440904,i have unblocked you eddie as i discussed on u...,0,"[146715, 80, 87, 20515, 26, 10684, 273, 80, 95...",i have unblocked you eddie as i discussed on u...,unblock eddie discuss user talkable accuse nil...


In [3]:
data = data.dropna()
data = data.reset_index()
data = data.drop(columns=["index"])

In [4]:
len(data)

440222

In [5]:
data["Label"] = data["Label"].astype(str)

data["Label"] = data["Label"].apply(lambda x: 1 if x == '1' else 0 if x == '0' else x)

data = data[data["Label"].isin([0, 1])]


In [6]:
len(data)

440215

In [7]:
data.head()

,Content,Label,Content_int,Cleaned_Content,Preprocessed_Content
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,...",denial of normal the con be asked to comment o...,denial normal con ask comment tragedy emotiona...
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22...",just by being able to tweet this insufferable ...,able tweet insufferable bullshit proves trump ...
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,...",that is retarded you too cute to be single tha...,retard cute single life
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,...",thought of a real badass mongol style declarat...,thought real badass mongol style declaration w...
4,afro american basho,1,"[146715, 46, 47, 48, 146714]",afro american basho,afro american basho


In [8]:
hate_speech_words = [
    'hate', 'violence', 'abuse', 'offend', 'discrimination', 'racism', 'oppression', 'bigot', 'ignorant', 
    'supremacist', 'feminazi', 'cunt', 'slut', 'whore', 'bitch', 'skank', 'thot', 'ho', 'gold digger', 
    'motherfucker', 'cockroach', 'asshole', 'dickhead', 'prick', 'scumbag', 'bastard', 'fuck', 'nigga', 
    'nigger', 'chink', 'gook', 'spic', 'wetback', 'cracker', 'honky', 'kike', 'wog', 'towelhead', 
    'sandnigger', 'faggot', 'fag', 'dyke', 'tranny', 'homo', 'sissy', 'fairy', 'pansy', 'sodomite', 
    'infidel', 'heathen', 'crusader', 'raghead', 'jihadist', 'taliban', 'beaner', 'gypsy', 'paki', 
    'wop', 'dago', 'mick', 'jap', 'yid', 'kaffir', 'cholo', 'zebra', 'white trash', 'redneck', 'hillbilly',
    'karen', 'snowflake', 'beta', 'incel', 'simp', 'boomer', 'npc', 'soyboy', 'mgtow', 'swine', 'trumpist', 
    'libtard', 'nazi', 'commie', 'sjw', 'ratchet', 'ghetto', 'redpill', 'based', 'chad', 'beta male', 
    'alpha male', 'cuck', 'cock', 'broke bitch', 'pussy ass', 'turd', 'feminist', 'terrorist', 'islamist'
]

word2vec_model = api.load("word2vec-google-news-300")

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')


In [9]:
def sentiment_analysis(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

def detect_hate_speech(text):
    return sum([1 for word in hate_speech_words if word in text.lower()])

def pos_tagging(text):
    doc = nlp(text)
    pos_tags = [token.pos_ for token in doc]
    return pos_tags

def get_word2vec_embedding(text):
    tokens = [token.text.lower() for token in nlp(text)]
    word_vectors = [word2vec_model[word] for word in tokens if word in word2vec_model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def lexical_diversity(text):
    tokens = [token.text for token in nlp(text)]
    return len(set(tokens)) / len(tokens) if len(tokens) > 0 else 0

def sentence_complexity(text):
    return len(re.findall(r'\.', text))

def get_sbert_embedding(text):
    return sbert_model.encode(text, convert_to_numpy=True)


In [10]:
# Feature extraction
data['sentiment'] = data['Preprocessed_Content'].apply(sentiment_analysis)
print('Sentiment analysis done')

data['hate_speech_count'] = data['Preprocessed_Content'].apply(detect_hate_speech)
print('Hate speech detection done')

data['pos_tags'] = data['Preprocessed_Content'].apply(pos_tagging)
print('POS tagging detection done')

data['word2vec'] = data['Preprocessed_Content'].apply(get_word2vec_embedding)
print('Word2Vec embedding done')

data['sbert_embedding'] = data['Preprocessed_Content'].apply(get_sbert_embedding)
print('SBERT embedding done')

data['lexical_diversity'] = data['Preprocessed_Content'].apply(lexical_diversity)
print('Lexical diversity done')

data['sentence_complexity'] = data['Preprocessed_Content'].apply(sentence_complexity)
print('Sentence complexity done')

Sentiment analysis done
Hate speech detection done
POS tagging detection done
Word2Vec embedding done
SBERT embedding done
Lexical diversity done
Sentence complexity done


In [11]:
data = data.drop_duplicates(subset=['Preprocessed_Content'])

### Drop unnecessary columns

In [12]:
data = data.drop(columns=['Content', 'Content_int', 'Cleaned_Content'])

In [13]:
data.columns

Index(['Label', 'Preprocessed_Content', 'sentiment', 'hate_speech_count',
       'pos_tags', 'word2vec', 'sbert_embedding', 'lexical_diversity',
       'sentence_complexity'],
      dtype='object')

In [14]:
len(data)

407364

### Save the dataset with extracted features

In [15]:
data.to_csv("Datasets/features.csv", index=False)